In [1]:
import pandas as pd 
df = pd.read_csv('fix_db.csv',low_memory=False)

In [2]:
df0 = df.drop_duplicates(subset=['commit_id'])
df0.shape

(15229, 12)

In [ ]:
# for _,row in df0.iterrows():
#     cm = row['label'].splitlines()[0]
#     if len(cm.split()) <= 1:
#         print(cm,row['local'],row['commit_id'])

In [4]:
result = list()
for cm in set(df['commit_id']):
    # cm 
    tmp = dict()
    message = ''
    summary = ''
    project = ''
    language = 'C'
    diffs = list()
    for _, row in df[df['commit_id']==cm].iterrows():
        tmp_diff = dict()
        if isinstance(row['new_path_file'],str):
            tmp_diff['positive_changed_file_name'] = row['new_path_file']
        else:
            if isinstance(row['old_path_file'],str):
                tmp_diff['positive_changed_file_name'] = row['old_path_file']
            else:
                tmp_diff['positive_changed_file_name'] = ''
        if isinstance(row['old_path_file'],str):
            tmp_diff['negative_changed_file_name'] = row['old_path_file']
        else:
            tmp_diff['negative_changed_file_name'] = tmp_diff['positive_changed_file_name']
        chunks = list()
        positive_changes = list()
        negative_changes = list()
        chunk_list = list()
        for line in row['diff'].splitlines():
            if line.startswith('+'):
                positive_changes.append(line[1:])
                chunk_list.append(line[1:])
            elif line.startswith('-'):
                negative_changes.append(line[1:])
                chunk_list.append(line[1:])
            else:
                if len(chunk_list) > 0:
                    chunks.append({'positive_changes':positive_changes,'negative_changes':negative_changes,'chunk_str':'\n'.join(chunk_list)})
                    chunk_list = list()
                    positive_changes = list()
                    negative_changes = list()
        tmp_diff['chunks'] = chunks
        diffs.append(tmp_diff)
        
        if len(project) <= 0:
            project = row['local'].split('/')[-1]
            message = row['label'].splitlines()[0]
            summary = message
    if len(message) > 0:
        result.append({
            'commit_id': cm,
            'message':message,
            'summary': summary,
            'language': language,
            'diffs' : diffs,
            'repo' : project
        })
len(result)

15229

In [6]:
import json
cms = list()
with open('test.jsonl') as f:
    for l in f.readlines():
        data = json.loads(l.strip())
        cm = data['commit_id']
        cms.append(cm)

In [7]:
test = list()
train = list()
for el in result:
    if el['commit_id'] in cms:
        test.append(el)
    else:
        train.append(el)
print(len(train),len(test))

12183 3046


In [18]:
15229 * 0.8

12183.2

In [50]:
train, test = result[:12183], result[12183:]

In [20]:
import gzip, codecs
def save_to_jsonl_gz(file_name, functions):
    with gzip.GzipFile(file_name, 'wb') as out_file:
        writer = codecs.getwriter('utf-8')
        for entry in functions:
            writer(out_file).write(json.dumps(entry))
            writer(out_file).write('\n')

In [51]:
save_to_jsonl_gz('train.jsonl.gz',train)
save_to_jsonl_gz('test.jsonl.gz',test)

In [ ]:
for el0 in result:
    # print(el['message'])
    for el in el0['diffs']:
        print(el['positive_changed_file_name'],el['negative_changed_file_name'])